In [ ]:
# ✅ Step 0: Install required package
!pip install -q transformers sentence-transformers

# ✅ Step 1: Import libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import numpy as np

# ✅ Step 2: Load models
qa_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# ✅ Step 3: Knowledge base
docs = [
    {
        "title": "When Life Gives You Tangerines",
        "description": "A 2025 South Korean drama set in Jeju Island, following the lifelong love story of Ae-sun and Gwan-sik, highlighting themes of perseverance and gratitude."
    },
    {
        "title": "Queen of Tears",
        "description": "A 2024 romantic drama about a chaebol heiress and her husband, achieving record-breaking ratings."
    },
    {
        "title": "The Glory",
        "description": "A 2023 revenge drama starring Song Hye-kyo as a woman seeking vengeance for her past."
    },
    {
        "title": "Extraordinary Attorney Woo",
        "description": "A 2022 series about an autistic lawyer navigating the legal world."
    },
    {
        "title": "My Mister",
        "description": "A 2018 drama depicting the bond between a middle-aged man and a young woman facing life's hardships."
    },
    {
        "title": "Crash Landing on You",
        "description": "A 2019 romance between a South Korean heiress and a North Korean officer."
    },
    {
        "title": "Reply 1988",
        "description": "A nostalgic 2015 series set in late 1980s Seoul, focusing on family and friendship."
    },
    {
        "title": "Mr. Sunshine",
        "description": "A 2018 historical drama set in the early 1900s, exploring Korea's fight for independence."
    },
    {
        "title": "Hospital Playlist",
        "description": "A 2020 medical drama following five doctors who have been friends since medical school."
    },
    {
        "title": "Signal",
        "description": "A 2016 crime thriller where detectives communicate across time to solve cold cases."
    },
    {
        "title": "Kingdom",
        "description": "A 2019 historical zombie thriller set in the Joseon dynasty."
    },
    {
        "title": "Vincenzo",
        "description": "A 2021 series about a Korean-Italian mafia lawyer taking on corrupt corporations."
    },
    {
        "title": "It's Okay to Not Be Okay",
        "description": "A 2020 romance addressing mental health through the story of a caregiver and a writer."
    },
    {
        "title": "Hotel del Luna",
        "description": "A 2019 fantasy drama about a hotel for ghosts, managed by an immortal woman."
    },
    {
        "title": "Itaewon Class",
        "description": "A 2020 series about an ex-convict opening a bar in Itaewon to challenge a food conglomerate."
    },
    {
        "title": "Start-Up",
        "description": "A 2020 drama focusing on young entrepreneurs in South Korea's startup ecosystem."
    },
    {
        "title": "Move to Heaven",
        "description": "A 2021 series about trauma cleaners uncovering untold stories of the deceased."
    },
    {
        "title": "Penthouse",
        "description": "A 2020-2021 drama depicting the lives of wealthy families in a luxury apartment complex."
    },
    {
        "title": "Sky Castle",
        "description": "A 2018 satirical series about elite families pushing their children into top universities."
    },
    {
        "title": "My Love from the Star",
        "description": "A 2013 fantasy romance between an alien and a top actress."
    },
    {
        "title": "Hometown Cha-Cha-Cha",
        "description": "A 2021 romantic comedy set in a seaside village, focusing on a dentist and a handyman."
    }
]

# ✅ Step 4: Embed all document texts
combined_texts = [f"{doc['title']}. {doc['description']}" for doc in docs]
doc_embeddings = embedder.encode(combined_texts, normalize_embeddings=True)

# ✅ Step 5: Define QA function (baseline, no context)
def answer_without_context(question):
    print("\n🧠 [No RAG] Running baseline model...")
    input_ids = qa_tokenizer(question, return_tensors='pt').input_ids
    output_ids = qa_model.generate(input_ids, max_length=64)
    return qa_tokenizer.decode(output_ids[0], skip_special_tokens=True)

# ✅ Step 6: Retrieve top-k docs
def retrieve_top_doc(question, k=2, return_indices=False):
    query_vec = embedder.encode(question, normalize_embeddings=True)
    scores = np.dot(doc_embeddings, query_vec)
    top_k_idx = np.argsort(scores)[::-1][:k]
    print("\n🔍 Top retrieved docs:")
    for i in top_k_idx:
        print(f"  - {docs[i]['title']}: {docs[i]['description']}")
    return top_k_idx if return_indices else [docs[i]['description'] for i in top_k_idx]

# ✅ Step 7: Generate answer using retrieved context
def answer_with_rag(question, k=2):
    top_doc_indices = retrieve_top_doc(question, k, return_indices=True)
    top_docs = [docs[i] for i in top_doc_indices]
    context = " ".join([f"{doc['title']}. {doc['description']}" for doc in top_docs])
    prompt = f"Context: {context}\n\nQuestion: {question}"
    print("\n🧠 [RAG] Constructed prompt for generation:")
    print(prompt)
    input_ids = qa_tokenizer(prompt, return_tensors='pt').input_ids
    output_ids = qa_model.generate(input_ids, max_length=100)
    return qa_tokenizer.decode(output_ids[0], skip_special_tokens=True)

# ✅ Step 8: Test question
#question = "Which Korean drama features a Korean-Italian mafia lawyer fighting against corrupt corporations?"
# question = "Which 2025 Korean drama title in Jeju Island tells a lifelong love story and became a major hit on Netflix?"
#question = "Which Korean drama features Song Hye-kyo as a woman who plots revenge after being bullied in school?"
question = "Which 2018 Korean drama satirizes elite families obsessively pushing their children into top universities?"
print("🧾 Question:", question)

print("============================================================")
answer_baseline = answer_without_context(question)
print("❌ Baseline Answer:", answer_baseline)

print("============================================================")
answer_rag = answer_with_rag(question)
print("✅ RAG Answer:", answer_rag)

🧾 Question: Which 2018 Korean drama satirizes elite families obsessively pushing their children into top universities?

🧠 [No RAG] Running baseline model...
❌ Baseline Answer: samsung

🔍 Top retrieved docs:
  - Sky Castle: A 2018 satirical series about elite families pushing their children into top universities.
  - Start-Up: A 2020 drama focusing on young entrepreneurs in South Korea's startup ecosystem.

🧠 [RAG] Constructed prompt for generation:
Context: Sky Castle. A 2018 satirical series about elite families pushing their children into top universities. Start-Up. A 2020 drama focusing on young entrepreneurs in South Korea's startup ecosystem.

Question: Which 2018 Korean drama satirizes elite families obsessively pushing their children into top universities?
✅ RAG Answer: Sky Castle
